# Instalight

The goal of the project is to provide a easy way to assemble and thus simulate the response of photonic integrated circuit, primarily using transfer matrices. Interactive widgets are used to enable real-time tuning of parameters and display of transmissions ect.

The following components are considered as fundamental: waveguides, directional couplers, linear phase shifters, multimode interferometer (MMI), and Y-branch.

Common building blocks, such as ring resonators and mach zehnder interferometer (MZI), are also available.

### global parameters

In [1]:
# Import libraries to be used
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from tqdm import tqdm
import scipy.constants as sc
import matplotlib as mpl
import scipy.io as io
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Defining the constant values
ng = 4
dndl = -2580642.76129          # 1/m
wg_loss = 360                  # dB/m
pn_loss = 2000                 # dB/m
pi = np.pi
c = sc.c                       # speed of light in vacuum

### functions for simulating fundamental components

In [3]:
def find_neff(lambda_c, ng=4, dndl=-2580642.76129):
    return dndl*lambda_c + ng
def phi(l, lambda_c):
    return 2*pi/lambda_c * find_neff(lambda_c, ng = 4.26, dndl = dndl)*l
def waveguide(l, lambda_c):
    phi = 2*pi/lambda_c * find_neff(lambda_c, ng = 4.26, dndl = dndl)*l
    return np.exp(-1j*phi) * 10**(-wg_loss*l/10)
def get_kappa_matrix(kappa):
    return np.matrix([[np.sqrt(1-kappa), 1j*np.sqrt(kappa)], 
                                  [1j*np.sqrt(kappa), np.sqrt(1-kappa)]])
# define kappa as through port power coupling ratio
def MMI_2x2(kappa = 0.5, phase = pi/2):
    return np.matrix([[np.sqrt(kappa), np.sqrt(1-kappa)*np.exp(1j*phase)],[np.sqrt(1-kappa)*np.exp(1j*phase),np.sqrt(kappa)]])

## Mach-Zehnder Interferometer (MZI)

<img src = 'MZI2.png' style = 'width:600px;height:400px'/>

In [4]:
# The function returns the transfer matrix of MZI
def MZI(kappa_1 = 0.1, kappa_2 = 0.7, arm_1 = 10e-6, arm_2 = 100e-6, wg_loss = 360, lambda_c = 1550e-9):
    phi_arm_1 = phi(l = arm_1, lambda_c=lambda_c)
    phi_arm_2 = phi(l = arm_2, lambda_c=lambda_c)
    
    kappa_1_matrix = get_kappa_matrix(kappa_1)
    mzi_arm_matrix = np.matrix([[np.exp(-1j*phi_arm_1)*10**(-wg_loss*arm_1/10), 0],
                                    [0,np.exp(-1j*phi_arm_2)*10**(-wg_loss*arm_2/10)]])
    kappa_2_matrix = get_kappa_matrix(kappa_2)
    
    mzi_matrix = kappa_2_matrix*mzi_arm_matrix*kappa_1_matrix
    return mzi_matrix

In [5]:
def plot_transmission_MZI(kappa1, kappa2, center_wavelength, wavelength_range, arm1, arm2):
    center_wavelength = center_wavelength*1e-09
    wavelength_range = wavelength_range*1e-09
    arm1 = arm1*1e-06
    arm2 = arm2*1e-06
    Start_wavelength = center_wavelength-wavelength_range
    Stop_wavelength = center_wavelength+wavelength_range
    Sweep_step = 1000
    wavelength = np.linspace(Start_wavelength, Stop_wavelength, Sweep_step)
    mzi_m = MZI(kappa_1 = kappa1, kappa_2 = kappa2, arm_1 = arm1, arm_2 = arm2, lambda_c = wavelength)
    plt.plot(wavelength*1e09, 20*np.log10(np.abs(mzi_m[0,0])**2))
    plt.title('$\kappa_1$ = {:.2f}, $\kappa_2$ = {:.2f}, Arm 1 = {:.1f} um, Arm 2 = {:.1f} um'.format(kappa1,kappa2,arm1*1e06,arm2*1e06))
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Transmission (dB)")
    plt.ylim(-50,0)
    plt.rcParams['figure.figsize'] = [12, 6]

In [6]:
interact(plot_transmission_MZI, center_wavelength = widgets.IntSlider(value=1550, min=1500, max=1600, step=1, description='Center Wavelength (nm):'),
                                       wavelength_range = widgets.IntSlider(value=50,    min=0,    max=100,  step=1, description='Wavelength Range (nm):'), 
                                       kappa1 = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='Kappa 1:'),
                                       kappa2 = widgets.FloatSlider(value=0.8, min=0, max=1, step=0.01, description='Kappa 2:'),
                                       arm1 = widgets.FloatSlider(value=60, min=5, max=200, step=0.1, description='Arm 1 (um):'),
                                       arm2 = widgets.FloatSlider(value=100, min=5, max=200, step=0.1, description='Arm 2 (um):'))

interactive(children=(FloatSlider(value=0.3, description='Kappa 1:', max=1.0, step=0.01), FloatSlider(value=0.…

<function __main__.plot_transmission_MZI(kappa1, kappa2, center_wavelength, wavelength_range, arm1, arm2)>

## Ring resonators

### Notch filter

<img src = 'Notch.png' style = 'width:500px;height:350px'/>

In [7]:
def notch_filter(lambda_c, kappa, l, input_1):
    waveguide_1 = waveguide(l, lambda_c)
    coefficient_m =  np.array([[-np.sqrt(1-kappa), 1.]
                              ,[-1., waveguide_1]])
    value_m = np.array([1j*np.sqrt(kappa)*input_1, 0.])
    solution = np.linalg.solve(coefficient_m, value_m)
    output_1 = np.sqrt(1-kappa)*input_1 + 1j*np.sqrt(kappa) * solution[0]
    return output_1

In [8]:
def plot_transmission_notch(kappa, center_wavelength, wavelength_range, l):
    center_wavelength = center_wavelength*1e-09
    wavelength_range = wavelength_range*1e-09
    l = l*1e-06
    Start_wavelength = center_wavelength-wavelength_range
    Stop_wavelength = center_wavelength+wavelength_range
    Sweep_step = 1000
    wavelength = np.linspace(Start_wavelength, Stop_wavelength, Sweep_step)
    results  = np.zeros(1000,dtype = 'complex_')
    for i in range(1000):
        results[i] = notch_filter(wavelength[i],kappa,l,1)
    plt.plot(wavelength*1e09, 20*np.log10(np.abs(results)**2))
    plt.title('$\kappa$ = {:.2f}, Waveguide = {:.1f} um'.format(kappa, l*1e06))
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Transmission (dB)")
    plt.ylim(-15,0)
    plt.rcParams['figure.figsize'] = [12, 6]

In [9]:
interact(plot_transmission_notch, center_wavelength = widgets.IntSlider(value=1550, min=1500, max=1600, step=1, description='Center Wavelength (nm):'),
                                       wavelength_range = widgets.IntSlider(value=25,    min=0,    max=100,  step=1, description='Wavelength Range (nm):'), 
                                       kappa = widgets.FloatSlider(value=0.01, min=0, max=1, step=0.1, description='Kappa'),
                                       l = widgets.FloatSlider(value=200, min=50, max=500, step=1, description='l (um):'))

interactive(children=(FloatSlider(value=0.01, description='Kappa', max=1.0), IntSlider(value=1550, description…

<function __main__.plot_transmission_notch(kappa, center_wavelength, wavelength_range, l)>

### Generalization: Ring with n directional couplers

Note: The order of coupler_list, waveguide_list, and input_list should be self-consistent (clockwise/counter clockwise direction). 1st waveguide should be the one encountered by the propogating wave when it leaves the 1st directional coupler.

In [10]:
# This function takes in a list of coupling coefficient, waveguide attenuation & phase shift coefficients
# to calculate the output at "throughput" ports given the specified input
def simple_ring(n, coupler_list, waveguide_list, input_list):
    coefficient_m = np.zeros((2*n, 2*n),dtype = 'complex_')
    value_m = np.zeros(2*n,dtype = 'complex_')
    output_list = np.zeros(n,dtype = 'complex_')
    for i in range(n):
        coefficient_m[i,i] = -np.sqrt(1-coupler_list[i])
        coefficient_m[i,n+i] = 1
        coefficient_m[2*n-1-i,i] = -1
        coefficient_m[n+i,n+i] = waveguide_list[i]
        value_m[i] = 1j*np.sqrt(coupler_list[i])*input_list[i]
    solution = np.linalg.solve(coefficient_m, value_m)
    for m in range(n):
        output_list[m] = np.sqrt(1-coupler_list[m])*input_list[m] + 1j*np.sqrt(coupler_list[m]) * solution[m]
    return output_list

### Special case: Add-drop filter

<img src = 'Add-Drop.png' style = 'width:700px;height:500px'/>

In [11]:
def plot_transmission_adddrop(kappa_1, kappa_2, central_wavelength, wavelength_range, l_1, l_2, input_1, input_2):
    # define the central wavelength, range of wavelength sweep, and number of steps in wavelength
    central_wavelength = central_wavelength*1e-09
    wavelength_range = wavelength_range*1e-09
    l_1 = l_1 * 1e-06
    l_2 = l_2 * 1e-06
    steps = 1000
    # define the number of directional couplers along the ring
    n = 2
    # define length of waveguides
    length_list = np.array([l_1,l_2])
    # define the coupling ratio of the directional couplers
    coupler_list = np.array([kappa_1,kappa_2])
    # define the power at input ports
    input_list = np.array([input_1,input_2])
    # initialize an array of wavelength and an array to store the power transmission at through ports
    wavelength = np.linspace(central_wavelength-wavelength_range, central_wavelength+wavelength_range, steps)
    result_1 = np.zeros(steps)
    result_2 = np.zeros(steps)
    for i in range(steps):
        waveguide_1 = waveguide(length_list[0], wavelength[i])
        waveguide_2 = waveguide(length_list[1], wavelength[i])
        waveguide_list = np.array([waveguide_1, waveguide_2])
        through_ports = simple_ring(n, coupler_list, waveguide_list, input_list)
        result_1[i] = np.abs(through_ports[0])**2
        result_2[i] = np.abs(through_ports[1])**2
    plt.plot(wavelength*1e09, 20*np.log10(result_1),label='Throughput port')
    plt.plot(wavelength*1e09, 20*np.log10(result_2),label='Drop port')
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Transmission (dB)")
    plt.ylim(-60,0)
    leg = plt.legend(loc='upper center')
    plt.show()

In [12]:
interact(plot_transmission_adddrop, central_wavelength = widgets.IntSlider(value=1550, min=1500, max=1600, step=1, description='Center Wavelength (nm):'),
                                       wavelength_range = widgets.IntSlider(value=10,    min=0,    max=100,  step=1, description='Wavelength Range (nm):'), 
                                       kappa_1 = widgets.FloatSlider(value=0.4, min=0, max=1, step=0.01, description='Kappa 1:'),
                                       kappa_2 = widgets.FloatSlider(value=0.6, min=0, max=1, step=0.01, description='Kappa 2:'),
                                       l_1 = widgets.FloatSlider(value=100, min=50, max=200, step=1, description='l_1 (um):'),
                                       l_2 = widgets.FloatSlider(value=100, min=50, max=200, step=1, description='l_2 (um):'),
                                       input_1 = widgets.FloatSlider(value=1, min=0, max=1, step=0.01, description='Input:'),
                                       input_2 = widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='Add:'))

interactive(children=(FloatSlider(value=0.4, description='Kappa 1:', max=1.0, step=0.01), FloatSlider(value=0.…

<function __main__.plot_transmission_adddrop(kappa_1, kappa_2, central_wavelength, wavelength_range, l_1, l_2, input_1, input_2)>

## 90 degree hybrid

<img src = '90_degree_hybrid.png' style = 'width:550px;height:400px'/>

<img src = 'hybrid_3.png' style = 'width:400px;height:200px'/>

<img src = 'hybrid_2.png' style = 'width:600px;height:200px'/>

In [13]:
# takes in 2x1 column vector [[E_Signal],[E_local_oscillator]] and return a 4x1 column vector of [[Q_p],[Q_n],[I_n],[I_p]]
def standard_hybrid(waveguide_list = np.array([np.exp(1j*0),np.exp(1j*0),np.exp(1j*0),np.exp(1j*0)])):
    # assume ideal power spliting
    kappa = 1/2
    # convert to 4x1 vector to match transfer matrices
    convert_m = np.zeros((4,2))
    convert_m[0,0] = 1; convert_m[1,1] = 1;
    # transfer matrix for Y-branch and 2x2 MMI
    split_m = np.zeros((4, 4),dtype = 'complex_')
    split_m[0,0] = 1; split_m[3,0] = np.exp(1j*pi/2); #2x2 MMI
    split_m[1,1] = 1; split_m[2,1] = 1; #Y-branch
    split_m = np.sqrt(kappa)*split_m
    # transfer matrix for bends
    bend_m = np.zeros((4, 4),dtype = 'complex_')
    for i in range(4):
        bend_m[i,i] = waveguide_list[i]
    # transfer matrix for Q & I channel
    mix_m = np.zeros((4, 4),dtype = 'complex_')
    mix_m[0:2,0:2] = MMI_2x2(); mix_m[2:,2:] = MMI_2x2();
    mix_m = np.sqrt(kappa)*mix_m
    hybrid_m = mix_m@bend_m@split_m@convert_m
    # ignore constant phase offset
    for i in range(4):
        hybrid_m[i] = hybrid_m[i]/hybrid_m[i,0]
    return hybrid_m
    

In [14]:
# takes in 2x1 column vector [[E_Signal],[E_local_oscillator]] and return a 4x1 column vector of [[Q_p],[Q_n],[I_n],[I_p]]
# waveguide_list order as labeled in the diagram
# kappa_list: signal MMI, Q channel MMI, Y-Branch, I channel MMI
def custom_hybrid(waveguide_list, kappa_list, MMI_phase = pi/2):
    # convert to 4x1 vector to match transfer matrices
    convert_m = np.zeros((4,2))
    convert_m[0,0] = 1; convert_m[1,1] = 1;
    # transfer matrix for Y-branch and 2x2 MMI
    split_m = np.zeros((4, 4),dtype = 'complex_')
    split_m[0,0] = np.sqrt(kappa_list[0]); split_m[3,0] = np.sqrt(1-kappa_list[0])*np.exp(1j*MMI_phase); #2x2 MMI
    split_m[1,1] = kappa_list[2]; split_m[2,1] = 1-kappa_list[2]; #Y-branch
    # transfer matrix for bends
    bend_m = np.zeros((4, 4),dtype = 'complex_')
    for i in range(4):
        bend_m[i,i] = waveguide_list[i]
    # transfer matrix for Q & I channel
    mix_m = np.zeros((4, 4),dtype = 'complex_')
    mix_m[0:2,0:2] = MMI_2x2(kappa = kappa_list[1], phase = MMI_phase); mix_m[2:,2:] = MMI_2x2(kappa = kappa_list[3], phase = MMI_phase);
    hybrid_m = mix_m@bend_m@split_m@convert_m
    # ignore constant phase offset
#     for i in range(4):
#         hybrid_m[i] = hybrid_m[i]/hybrid_m[i,0]
    return hybrid_m

In [15]:
def random_signal(n,noise_level):
    I = np.array([1,-1])
    Q = np.array([1j,-1j])
    s_re = np.random.choice(I,n)
    s_imag = np.random.choice(Q,n)
    pure_signal = s_re+s_imag
    noise = np.random.uniform(-1*noise_level,noise_level,n) + np.random.uniform(-1*noise_level,noise_level,n)*1j
    noisy_signal = pure_signal + noise
    return noisy_signal

In [16]:
def plot_constellation_hybrid(n_signal, noise_level, l1, l2, l3, l4, k1, k2, k3, k4, wavelength, MMI_phase):
    E_signal = random_signal(n_signal,noise_level)
    E_LO = 1
    length_list = 1e-06 * np.array([l1,l2,l3,l4])
    kappa_list = [k1,k2,k3,k4]
    wavelength = wavelength*1e-09
    MMI_phase = MMI_phase*pi
    waveguide_list = np.zeros(4,dtype = 'complex_')
    phi_list = np.zeros(4)
    Loss = 0; CMRR_i = 0; Imbalance_i = 0; CMRR_q = 0; Imbalance_q = 0;
    for i in range(4):
        waveguide_list[i] = waveguide(length_list[i],wavelength)
        phi_list[i] = phi(length_list[i],wavelength)
    plt.plot(1/np.sqrt(2),1/np.sqrt(2),'r+',mew=3, ms=30)
    plt.plot(1/np.sqrt(2),-1/np.sqrt(2),'r+',mew=3, ms=30)
    plt.plot(-1/np.sqrt(2),1/np.sqrt(2),'r+',mew=3, ms=30)
    plt.plot(-1/np.sqrt(2),-1/np.sqrt(2),'r+',mew=3, ms=30)
    for i in range(n_signal):
        input_vector = np.array([[E_signal[i]],[E_LO]])
        quadrature_states = custom_hybrid(waveguide_list, kappa_list, MMI_phase)@input_vector
        state_power = np.abs(quadrature_states)**2
        state_power_1 = np.array([state_power[0,0],state_power[1,0],state_power[2,0],state_power[3,0]])
#         print(state_power_1)
        sorted_power = sorted(state_power_1)
        P = np.array([sorted_power[0],sorted_power[1]])
        N = np.array([sorted_power[2],sorted_power[3]])
        Q = state_power[0,0]-state_power[1,0]
        I = state_power[3,0]-state_power[2,0]  
        plt.plot(I, Q, 'b.',ms = 5)
        Loss = Loss + 10*np.log10(np.abs(E_signal[i])**2+1)-10*np.log10(state_power[0,0]+state_power[1,0]+state_power[2,0]+state_power[3,0])
#         print(np.abs(E_signal[i])**2,state_power[0,0]+state_power[1,0]+state_power[2,0]+state_power[3,0])
#         state_power.sort()
#         CMRR_i = CMRR_i + 20*np.log10(np.abs((state_power[0]-state_power[1])/(state_power[0]+state_power[1])))
#         CMRR_q = CMRR_q + 20*np.log10(np.abs((state_power[2]-state_power[3])/(state_power[2]+state_power[3])))
#         Imbalance_i = Imbalance_i + 10*np.log10(state_power[0]/state_power[1])
#         Imbalance_q = Imbalance_q + 10*np.log10(state_power[2]/state_power[3])
#         CMRR_i = CMRR_i + 20*np.log10(np.abs((state_power[3,0]-state_power[2,0])/(state_power[3,0]+state_power[2,0])))
#         CMRR_q = CMRR_q + 20*np.log10(np.abs((state_power[0,0]-state_power[1,0])/(state_power[0,0]+state_power[1,0])))
#         Imbalance_i = Imbalance_i + 10*np.log10(state_power[3,0]/state_power[2,0])
#         Imbalance_q = Imbalance_q + 10*np.log10(state_power[0,0]/state_power[1,0])
        CMRR_i = CMRR_i + 20*np.log10(np.abs((P[0]-P[1])/(P[0]+P[1])))
        CMRR_q = CMRR_q + 20*np.log10(np.abs((N[0]-N[1])/(N[0]+N[1])))
        Imbalance_i = Imbalance_i + 10*np.log10(P[0]/P[1])
        Imbalance_q = Imbalance_q + 10*np.log10(N[0]/N[1])
    phase_error = (0.5*pi - (phi_list[1]-phi_list[0]) - (phi_list[3]-phi_list[2]) - MMI_phase)/pi
    Loss = Loss / n_signal; CMRR_i = CMRR_i / n_signal; Imbalance_i = Imbalance_i / n_signal; CMRR_q = CMRR_q / n_signal; Imbalance_q = Imbalance_q / n_signal;
    str2display = "Loss: "+"{:.3f}".format(Loss) + ' dB\n' + "CMRR_i: "+ "{:.3f}".format(CMRR_i) + ' dB\n' + "CMRR_q: "+ "{:.3f}".format(CMRR_q) + ' dB\n' + "Imbalance_i: "+ "{:.3f}".format(Imbalance_i) + ' dB\n' + "Imbalance_q: "+ "{:.3f}".format(Imbalance_q) + ' dB\n' + "Phase Error: "+ "{:.3f}".format(phase_error) + ' pi'
    plt.annotate(str2display,xy = (0.8,1.2))
    plt.ylabel('Quadrature Phase')
    plt.xlabel('In Phase')
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    plt.rcParams['figure.figsize'] = [8, 8]
    plt.show()

In [17]:
style = {'description_width': 'initial'}
interact(plot_constellation_hybrid, n_signal = widgets.IntSlider(value=100, min=50, max=500, step=10,               description='Signal Number:  ',style=style),
                                       noise_level = widgets.FloatSlider(value=0.1,    min=0,    max=1,  step=0.01, description='Noise Level:    ',style=style), 
                                       l1 = widgets.FloatSlider(value=10, min=1, max=50, step=0.1,                  description='Wg 1(um):',style=style),
                                       l2 = widgets.FloatSlider(value=10, min=1, max=50, step=0.1,                  description='Wg 2(um):',style=style),
                                       l3 = widgets.FloatSlider(value=10, min=1, max=50, step=0.1,                  description='Wg 3(um):',style=style),
                                       l4 = widgets.FloatSlider(value=10, min=1, max=50, step=0.1,                  description='Wg 4(um):',style=style),
                                       k1 = widgets.FloatSlider(value=0.5, min=0.4, max=0.6, step=0.01,             description='Kappa 1:        ',style=style),
                                       k2 = widgets.FloatSlider(value=0.5, min=0.4, max=0.6, step=0.01,             description='Kappa 2:        ',style=style),
                                       k3 = widgets.FloatSlider(value=0.5, min=0.4, max=0.6, step=0.01,             description='Kappa 3:        ',style=style),
                                       k4 = widgets.FloatSlider(value=0.5, min=0.4, max=0.6, step=0.01,             description='Kappa 4:        ',style=style),
                                       wavelength = widgets.FloatSlider(value=1550, min=1500, max=1600, step=0.01,  description='Wavelength(nm): ',style=style),
                                       MMI_phase = widgets.FloatSlider(value=0.5, min=0.4, max=0.6, step=0.01,      description='MMI_phase(Pi):  ',style=style))

interactive(children=(IntSlider(value=100, description='Signal Number:  ', max=500, min=50, step=10, style=Sli…

<function __main__.plot_constellation_hybrid(n_signal, noise_level, l1, l2, l3, l4, k1, k2, k3, k4, wavelength, MMI_phase)>

<img src = 'hybrid.png' style = 'width:550px;height:400px'/>